In [ ]:
# @title #  ZeroChat
# @markdown Create a Gradio chat space using a free ZeroGPU for a given model like [AlphaMonarch-7B](https://huggingface.co/spaces/mlabonne/AlphaMonarch-7B) or [Beyonder-4x7B-v3](https://huggingface.co/spaces/mlabonne/Beyonder-4x7B-v3).

# @markdown To use a ZeroGPU, you can request to join [this organisation](https://huggingface.co/zero-gpu-explorers) on Hugging Face.

# @markdown ---

!pip install -qqq huggingface_hub

from huggingface_hub import HfApi, hf_hub_download
from google.colab import runtime
import random

# @markdown ## 🧠  Model

# Input variables
MODEL_ID = "mlabonne/AlphaMonarch-7B" # @param {type:"string"}
CHAT_TEMPLATE = "Auto" # @param ["Auto", "ChatML", "Mistral Instruct"]
DESCRIPTION = "[AlphaMonarch-7B](https://huggingface.co/mlabonne/AlphaMonarch-7B) is a new DPO merge that retains all the reasoning abilities of the very best merges and significantly improves its conversational abilities. Kind of the best of both worlds in a 7B model. 🎉" # @param {type:"string"}
CONTEXT_LENGTH = 8196 # @param {type:"number"}
SPACE_COLOR1 = "purple" # @param ["red", "yellow", "green", "blue", "indigo", "purple", "pink", "gray"]
SPACE_COLOR2 = "yellow" # @param ["red", "yellow", "green", "blue", "indigo", "purple", "pink", "gray"]
GRADIO_COLOR = "amber" # @param ["slate", "gray", "zinc", "neutral", "stone", "red", "orange", "amber", "yellow", "lime", "green", "emerald", "teal", "cyan", "sky", "blue", "indigo", "violet", "purple", "fuchsia", "pink", "rose"]
EMOJI = "\uD83D\uDC51" # @param {type:"string"}

# @markdown ---

# @markdown ## 🤗 Hugging Face
# @markdown `HF_TOKEN` corresponds to the name of the secret that stores your [Hugging Face access token](https://huggingface.co/settings/tokens) in Colab.

USERNAME = "mlabonne" # @param {type:"string"}
HF_TOKEN = "HF_TOKEN" # @param {type:"string"}

from_id = "mlabonne/Beyonder-4x7B-v3"
model_name = MODEL_ID.split("/")[-1]
repo_id = f"{USERNAME}/{model_name}"
EMOJI = EMOJI.encode('utf-16', 'surrogatepass').decode('utf-16')

# Create space
api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="space", space_sdk="gradio", exist_ok=True)
api.request_space_hardware(repo_id=repo_id, hardware="zero-a10g")

# Add environment variables
api.add_space_variable(repo_id=repo_id, key="MODEL_ID", value=MODEL_ID)
api.add_space_variable(repo_id=repo_id, key="CHAT_TEMPLATE", value=CHAT_TEMPLATE)
api.add_space_variable(repo_id=repo_id, key="DESCRIPTION", value=DESCRIPTION)
api.add_space_variable(repo_id=repo_id, key="CONTEXT_LENGTH", value=str(CONTEXT_LENGTH))
api.add_space_variable(repo_id=repo_id, key="COLOR", value=GRADIO_COLOR)
api.add_space_variable(repo_id=repo_id, key="EMOJI", value=EMOJI)

# Clone template repo
!git clone https://huggingface.co/spaces/{repo_id}

# Download files
hf_hub_download(repo_id=from_id, repo_type="space", local_dir=model_name, filename="app.py")
hf_hub_download(repo_id=from_id, repo_type="space", local_dir=model_name, filename="requirements.txt")
hf_hub_download(repo_id=from_id, repo_type="space", local_dir=model_name, filename=".gitattributes")

# Create content
readme = f"""---
title: {model_name}
emoji: {EMOJI}
colorFrom: {SPACE_COLOR1}
colorTo: {SPACE_COLOR2}
sdk: gradio
sdk_version: 4.23.0
app_file: app.py
pinned: false
license: apache-2.0
header: mini
---

Check out the configuration reference at https://huggingface.co/docs/hub/spaces-config-reference"""

# Save README.md
with open(f"{model_name}/README.md", 'w') as file:
    file.write(readme)

# Upload files
api.upload_folder(repo_id=repo_id, repo_type="space", folder_path=model_name)